Importing required packages for the project

In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

import sys
import warnings
import datetime as dt
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

: 

In [ ]:
from math import floor,ceil,sqrt
from pmdarima.arima import auto_arima
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from prophet import Prophet
from sklearn.metrics import r2_score

: 

In [ ]:
if not sys.warnoptions:
    warnings.simplefilter("ignore")


: 

Loading the data for the prediction

In [ ]:
#Replace with your directory of MSFT.csv file
df = pd.read_csv("C:/Users/Username/MSFT.csv")
df.head(10)

: 

Data preprocessing, changed the format of date column from YYYY-MM-DD to MM/DD/YYYY HH:MM:SS in excel.

In [ ]:
#Replace with your directory of MSFT.csv file
df=pd.read_csv("C:/Users/Username/MSFT.csv")
df

: 

Data preprocessing, changed the format of date column from YYYY-MM-DD to MM/DD/YYYY HH:MM:SS, completing the action with pandas and indexing with date. Plotting the dataset.

In [ ]:
df['Date'] = pd.to_datetime(df.Date,format='%m/%d/%Y %H:%M:%S')
df.index = df['Date']
plt.figure(figsize=(14,7))
plt.plot(df['Close'], label='Close Price history',color='r')
plt.xlabel('Date',size=20)
plt.ylabel('Stock Price',size=18)
plt.title('Stock Price of Microsoft over the Years',size=23)

: 

In [ ]:
df.dtypes

: 

In [ ]:
df.isnull().sum()

: 

Moving average model for comparision purpose. RMSE and r2 score is used to evaluate the model performance. The prediction is plotted along with the actual values. The model is trained using the interquartile range in the data and tested against the fourth quartile 

In [ ]:
def Moving_Average_Prediction(df):
    shape=df.shape[0]
    df_new=df[['Close']]
    df_new.head()
    train__data_set=df_new.iloc[ceil(shape*0.25):ceil(shape*0.80)]
    valid__data_set=df_new.iloc[ceil(shape*0.75):]
    print("***** Microsoft Stock Price Prediction Using Moving Averages*********")
    print('Shape of Training Data Set of Microsft Stock Prices',train__data_set.shape)
    print('Shape of Validating Data Set of Microsft Stock Prices',valid__data_set.shape)
    preds = []
    for i in range(0,valid__data_set.shape[0]):
        a = train__data_set['Close'][len(train__data_set)-valid__data_set.shape[0]+i:].sum() + sum(preds)
        b = a/(valid__data_set.shape[0])
        preds.append(b)
    RMS=np.sqrt(np.mean(np.power((np.array(valid__data_set['Close'])-preds),2)))
    r2= r2_score(valid__data_set,preds)
    print('(Root Mean Square Error) RMSE value on validation set:',RMS)
    print('(R2 Score)R2 value on validation set:',r2)
    valid__data_set['Predictions'] = preds
    plt.plot(train__data_set['Close'])
    plt.plot(valid__data_set[['Close', 'Predictions']])
    plt.xlabel('Date',size=18)
    plt.ylabel('Stock Price',size=18)
    plt.title('Stock Price Prediction by Moving Averages',size=18)
    plt.legend(['Model Training Data','Actual Data','Predicted Data'])

: 

In [ ]:
Moving_Average_Prediction(df)

: 

The project proposal LSTM model for predicting stock price data.  RMSE and r2 score is used to evaluate the model performance. The prediction is plotted along with the actual values. The model is trained using near interquartile range in the data and tested against the fourth quartile. It is trainned for one epoch by keeping batch size 1.

In [ ]:
def LSTM_Prediction(df):
    shape=df.shape[0]
    df_new=df[['Close']]
    df_new.head()
    dataset = df_new.values
    train=df_new[ceil(shape*0.30):ceil(shape*0.80)]
    valid=df_new[ceil(shape*0.75):]
    print("******** Microsoft STOCK PRICE PREDICTION BY LONG SHORT TERM MEMORY (LSTM) *******")
    print('Shape of Training data Set',train.shape)
    print('Shape of Validation data Set',valid.shape)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)
    X_train, Y_train = [], []
    for i in range(40,len(train)):
        X_train.append(scaled_data[i-40:i,0])
        Y_train.append(scaled_data[i,0])
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    Y_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
    Model = Sequential()
    Model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
    Model.add(LSTM(units=50))
    Model.add(Dense(1))
    Model.compile(loss='mean_squared_error', optimizer='adam')
    Model.fit(X_train, Y_train, epochs=1, batch_size=2, verbose=2)
    inputs = df_new[len(df_new) - len(valid) - 40:].values
    inputs = inputs.reshape(-1,1)
    inputs  = scaler.transform(inputs)
    X_test = []
    for i in range(40,inputs.shape[0]):
        X_test.append(inputs[i-40:i,0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
    closing_price = Model.predict(X_test)
    closing_price = scaler.inverse_transform(closing_price)
    RMS=np.sqrt(np.mean(np.power((valid-closing_price),2)))
    r2 = r2_score(valid-closing_price ,closing_price )
    print('(R2 score)R2 value on validation set:', r2)
    print('(Root Neab Square Error) RMSE value on validation set:',RMS)
    valid['Predictions'] = closing_price
    plt.plot(train['Close'])
    plt.plot(valid[['Close','Predictions']])
    plt.xlabel('Date',size=20)
    plt.ylabel('Stock Price',size=20)
    plt.title('Microsoft Stock Price Prediction by Long Short Term Memory (LSTM)',size=20)
    plt.legend(['Model Training Data','Actual Data','Predicted Data'])

: 

In [ ]:
LSTM_Prediction(df)

: 

Moving average performance metrices :
    RMSE:48.88457740518917
    r2:-1.0234099920313238

LSTM performance metrices :
    RMSE:7.440627
    r2:-82.5785420245649